In [1]:
import akshare as ak
import shutup

shutup.please()

sina_futures = ak.futures_display_main_sina()
sina_futures

/Users/wangtianzhu/WebstormProjects/aphrodite/venv/lib/python3.9/site-packages/akshare/__init__.py:2609: UserWarning: 为了支持更多特性，请将 Pandas 升级到 2.1.0 及以上版本！
  warnings.warn(


,symbol,exchange,name
0,V0,dce,PVC连续
1,P0,dce,棕榈油连续
2,B0,dce,豆二连续
3,M0,dce,豆粕连续
4,I0,dce,铁矿石连续
...,...,...,...
70,IC0,cffex,中证500指数期货连续
71,TS0,cffex,2年期国债期货连续
72,IM0,cffex,中证连续指数期货连续
73,SI0,gfex,工业硅连续


In [9]:
PVC_data = ak.futures_main_sina(symbol='AG0')
PVC_data

,日期,开盘价,最高价,最低价,收盘价,成交量,持仓量,动态结算价
0,2012-05-10,6190,6196,6082,6142,259996,33726,6126
1,2012-05-11,6110,6115,6000,6002,206804,55020,6056
2,2012-05-14,6040,6045,5960,5977,207668,57028,5999
3,2012-05-15,5895,5910,5711,5843,445150,82238,5813
4,2012-05-16,5750,5786,5643,5647,599186,108376,5704
...,...,...,...,...,...,...,...,...
2808,2023-11-20,5940,5940,5860,5868,846816,500990,5889
2809,2023-11-21,5838,5878,5816,5875,841583,486628,5845
2810,2023-11-22,5855,5916,5855,5891,830459,493146,5893
2811,2023-11-23,5897,5904,5844,5877,833145,483511,5870


In [3]:
PVC_data.rename(columns={'日期': 'date', '开盘价': 'open', '收盘价': 'close', '最高价': 'high', '最低价': 'low', '成交量': 'volume', '持仓量': 'inventory'}, inplace=True)
PVC_data.drop(columns=['动态结算价'], inplace=True)
PVC_data

,date,open,high,low,close,volume,inventory
0,2009-05-25,6575,6630,6435,6490,107764,12278
1,2009-05-26,6485,6540,6445,6460,38290,10562
2,2009-05-27,6500,6535,6460,6485,34002,10346
3,2009-06-01,6555,6595,6520,6570,26178,10352
4,2009-06-02,6590,6590,6545,6555,13496,9388
...,...,...,...,...,...,...,...
3515,2023-11-20,6068,6094,6005,6014,870223,770701
3516,2023-11-21,6025,6032,5912,5915,1046568,811110
3517,2023-11-22,5920,5947,5874,5906,716774,814395
3518,2023-11-23,5890,5898,5781,5815,944831,840537


In [8]:
sina_futures.sort_values(by='symbol')


,symbol,exchange,name
12,A0,dce,豆一连续
54,AG0,shfe,白银连续
46,AL0,shfe,铝连续
64,AO0,shfe,氧化铝连续
36,AP0,czce,苹果连续
...,...,...,...
0,V0,dce,PVC连续
25,WH0,czce,强麦连续
52,WR0,shfe,线材连续
10,Y0,dce,豆油连续


In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd
import pickle
import threading
import sys


class FOMC(object):
    """
    A convenient class for extracting meeting minutes from the FOMC website
    Example Usage:
        fomc = FOMC()
        df = fomc.get_statements()
        fomc.pickle("./df_minutes.pickle")
    """

    def __init__(self, base_url='https://www.federalreserve.gov',
                 calendar_url='https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm',
                 historical_date=2011,
                 verbose=True,
                 max_threads=10):

        self.base_url = base_url
        self.calendar_url = calendar_url
        self.df = None
        self.links = None
        self.dates = None
        self.articles = None
        self.verbose = verbose
        self.HISTORICAL_DATE = historical_date
        self.MAX_THREADS = max_threads

    def _get_links(self, from_year):
        """
        private function that sets all the links for the FOMC meetings from the giving from_year
        to the current most recent year
        """
        if self.verbose:
            print("Getting links...")
        self.links = []
        fomc_meetings_socket = urlopen(self.calendar_url)
        soup = BeautifulSoup(fomc_meetings_socket, 'html.parser')

        statements = soup.find_all('a', href=re.compile('^/newsevents/pressreleases/monetary\d{8}a.htm'))
        self.links = [statement.attrs['href'] for statement in statements]

        if from_year <= self.HISTORICAL_DATE:
            for year in range(from_year, self.HISTORICAL_DATE + 1):
                fomc_yearly_url = self.base_url + '/monetarypolicy/fomchistorical' + str(year) + '.htm'
                fomc_yearly_socket = urlopen(fomc_yearly_url)
                soup_yearly = BeautifulSoup(fomc_yearly_socket, 'html.parser')
                statements_historical = soup_yearly.findAll('a', text='Statement')
                for statement_historical in statements_historical:
                    self.links.append(statement_historical.attrs['href'])

    def _date_from_link(self, link):
        date = re.findall('[0-9]{8}', link)[0]
        if date[4] == '0':
            date = "{}/{}/{}".format(date[:4], date[5:6], date[6:])
        else:
            date = "{}/{}/{}".format(date[:4], date[4:6], date[6:])
        return date

    def _add_article(self, link, index=None):
        """
        adds the related article for 1 link into the instance variable
        index is the index in the article to add to. Due to concurrent
        processing, we need to make sure the articles are stored in the
        right order
        """
        if self.verbose:
            sys.stdout.write(".")
            sys.stdout.flush()

        # date of the article content
        self.dates.append(self._date_from_link(link))
        statement_socket = urlopen(self.base_url + link)
        statement = BeautifulSoup(statement_socket, 'html.parser')
        paragraphs = statement.findAll('p')
        self.articles[index] = "\n\n".join([paragraph.get_text().strip() for paragraph in paragraphs])

    def _get_articles_multi_threaded(self):
        """
        gets all articles using multi-threading
        """
        if self.verbose:
            print("Getting articles - Multi-threaded...")

        self.dates, self.articles = [], [''] * len(self.links)
        jobs = []
        # initiate and start threads:
        index = 0
        while index < len(self.links):
            if len(jobs) < self.MAX_THREADS:
                t = threading.Thread(target=self._add_article, args=(self.links[index], index,))
                jobs.append(t)
                t.start()
                index += 1
            else:  # wait for threads to complete and join them back into the main thread
                t = jobs.pop(0)
                t.join()
        for t in jobs:
            t.join()

        for row in range(len(self.articles)):
            self.articles[row] = self.articles[row].strip()

    def get_statements(self, from_year=1994):
        """
        Returns a Pandas DataFrame of meeting minutes with the date as the index
        uses a date range of from_year to the most current

        Input from_year is ignored if it is within the last 5 years as this is meant for
        parsing much older years
        """
        self._get_links(from_year)
        print("There are", len(self.links), 'statements')
        self._get_articles_multi_threaded()

        self.df = pd.DataFrame(self.articles, index=pd.to_datetime(self.dates)).sort_index()
        self.df.columns = ['statements']
        return self.df

    def pick_df(self, filename="../data/minutes.pickle"):
        if filename:
            if self.verbose:
                print("Writing to", filename)
            with open(filename, "wb") as output_file:
                pickle.dump(self.df, output_file)

# Example Usage
fomc = FOMC()
df = fomc.get_statements()
df

Getting links...


HTTPError: HTTP Error 403: Forbidden